# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [2]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [3]:
# load in the data
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')


data = pd.read_excel("./DIAS Information Levels - Attributes 2017.xlsx")
data2 = pd.read_excel("./DIAS Attributes - Values 2017.xlsx")

keep_columns = list(set(customers.columns) & set(data.Attribute))
len(keep_columns)


customers = customers[keep_columns]
azdias = azdias[keep_columns]

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
data2_2 = data2.dropna(subset = ["Attribute"])
data2


,Unnamed: 0,Attribute,Description,Value,Meaning
0,NaN,AGER_TYP,best-ager typology,-1,unknown
1,NaN,NaN,NaN,0,no classification possible
2,NaN,NaN,NaN,1,passive elderly
3,NaN,NaN,NaN,2,cultural elderly
4,NaN,NaN,NaN,3,experience-driven elderly
5,NaN,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,NaN,NaN,NaN,1,< 30 years
7,NaN,NaN,NaN,2,30 - 45 years
8,NaN,NaN,NaN,3,46 - 60 years
9,NaN,NaN,NaN,4,> 60 years


In [4]:
data.Attribute[0:50]

0                  AGER_TYP
1      ALTERSKATEGORIE_GROB
2                 ANREDE_KZ
3             CJT_GESAMTTYP
4         FINANZ_MINIMALIST
5             FINANZ_SPARER
6          FINANZ_VORSORGER
7            FINANZ_ANLEGER
8     FINANZ_UNAUFFAELLIGER
9          FINANZ_HAUSBAUER
10                FINANZTYP
11              GEBURTSJAHR
12          GFK_URLAUBERTYP
13         GREEN_AVANTGARDE
14               HEALTH_TYP
15      LP_LEBENSPHASE_FEIN
16      LP_LEBENSPHASE_GROB
17          LP_FAMILIE_FEIN
18          LP_FAMILIE_GROB
19           LP_STATUS_FEIN
20           LP_STATUS_GROB
21         NATIONALITAET_KZ
22    PRAEGENDE_JUGENDJAHRE
23           RETOURTYP_BK_S
24                SEMIO_SOZ
25                SEMIO_FAM
26                SEMIO_REL
27                SEMIO_MAT
28               SEMIO_VERT
29               SEMIO_LUST
30                SEMIO_ERL
31               SEMIO_KULT
32                SEMIO_RAT
33               SEMIO_KRIT
34                SEMIO_DOM
35               SEM

In [5]:
data

,Unnamed: 0,Information level,Attribute,Description,Additional notes
0,NaN,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,NaN,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,NaN,NaN,ANREDE_KZ,gender,NaN
3,NaN,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,NaN,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...
5,NaN,NaN,FINANZ_SPARER,financial typology: money saver,NaN
6,NaN,NaN,FINANZ_VORSORGER,financial typology: be prepared,NaN
7,NaN,NaN,FINANZ_ANLEGER,financial typology: investor,NaN
8,NaN,NaN,FINANZ_UNAUFFAELLIGER,financial typology: unremarkable,NaN
9,NaN,NaN,FINANZ_HAUSBAUER,financial typology: main focus is the own house,NaN


In [6]:
# Be sure to add in a lot more cells (both markdown and code) to document your
# approach and findings!


In [7]:
print(azdias.head())
print(azdias.info())

   KBA13_OPEL  KBA05_KW1  KBA13_KMH_180  KBA13_HALTER_25  \
0         NaN        NaN            NaN              NaN   
1         3.0        1.0            2.0              3.0   
2         2.0        3.0            2.0              3.0   
3         2.0        3.0            2.0              2.0   
4         3.0        3.0            1.0              3.0   

   D19_GESAMT_ONLINE_DATUM  RETOURTYP_BK_S  FINANZ_VORSORGER  \
0                       10             5.0                 3   
1                       10             1.0                 2   
2                       10             3.0                 1   
3                       10             2.0                 5   
4                        1             5.0                 4   

   KBA13_SEG_MITTELKLASSE  KBA13_VORB_1_2  KBA05_SEG5          ...           \
0                     NaN             NaN         NaN          ...            
1                     3.0             3.0         2.0          ...            
2                

In [8]:
azdias.describe()

,KBA13_OPEL,KBA05_KW1,KBA13_KMH_180,KBA13_HALTER_25,D19_GESAMT_ONLINE_DATUM,RETOURTYP_BK_S,FINANZ_VORSORGER,KBA13_SEG_MITTELKLASSE,KBA13_VORB_1_2,KBA05_SEG5,...,D19_VERSI_DATUM,KBA05_SEG2,D19_TELKO_OFFLINE_DATUM,KBA05_ALTER2,ALTERSKATEGORIE_GROB,KONSUMNAEHE,D19_GESAMT_ONLINE_QUOTE_12,KBA05_KRSHERST1,LP_LEBENSPHASE_FEIN,KBA13_SEG_WOHNMOBILE
count,785421.000000,757897.000000,785421.000000,785421.000000,891221.000000,886367.000000,891221.000000,785421.000000,785421.000000,757897.000000,...,891221.000000,757897.000000,891221.000000,757897.000000,891221.000000,817252.000000,634108.000000,757897.000000,886367.000000,785421.000000
mean,2.971935,3.093550,2.932968,2.945092,7.680381,3.419630,3.401106,3.058855,2.916887,1.597992,...,9.142563,3.098730,9.828039,3.149416,2.777398,3.018452,3.560952,3.049426,14.622637,2.538733
std,1.054774,1.376706,0.970028,1.036591,3.039867,1.417741,1.322134,1.026028,0.961899,1.567611,...,1.911186,1.332421,0.745852,1.338676,1.068775,1.550312,4.658538,1.308095,12.616883,1.411366
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
25%,2.000000,2.000000,2.000000,2.000000,5.000000,2.000000,3.000000,2.000000,2.000000,1.000000,...,9.000000,2.000000,10.000000,2.000000,2.000000,2.000000,0.000000,2.000000,4.000000,2.000000
50%,3.000000,3.000000,3.000000,3.000000,10.000000,3.000000,3.000000,3.000000,3.000000,1.000000,...,10.000000,3.000000,10.000000,3.000000,3.000000,3.000000,0.000000,3.000000,11.000000,3.000000
75%,4.000000,4.000000,4.000000,4.000000,10.000000,5.000000,5.000000,4.000000,3.000000,2.000000,...,10.000000,4.000000,10.000000,4.000000,4.000000,4.000000,10.000000,4.000000,27.000000,3.000000
max,5.000000,9.000000,5.000000,5.000000,10.000000,5.000000,5.000000,5.000000,5.000000,9.000000,...,10.000000,9.000000,10.000000,9.000000,9.000000,7.000000,10.000000,9.000000,40.000000,5.000000


In [9]:
print(customers.head())
print(customers.info())

   KBA13_OPEL  KBA05_KW1  KBA13_KMH_180  KBA13_HALTER_25  \
0         3.0        3.0            2.0              1.0   
1         NaN        NaN            NaN              NaN   
2         1.0        3.0            3.0              3.0   
3         1.0        3.0            3.0              4.0   
4         4.0        3.0            4.0              2.0   

   D19_GESAMT_ONLINE_DATUM  RETOURTYP_BK_S  FINANZ_VORSORGER  \
0                       10             5.0                 5   
1                        9             NaN                 5   
2                       10             5.0                 5   
3                       10             3.0                 5   
4                        1             5.0                 4   

   KBA13_SEG_MITTELKLASSE  KBA13_VORB_1_2  KBA05_SEG5          ...           \
0                     3.0             3.0         3.0          ...            
1                     NaN             NaN         NaN          ...            
2                

In [10]:
customers.describe()

,KBA13_OPEL,KBA05_KW1,KBA13_KMH_180,KBA13_HALTER_25,D19_GESAMT_ONLINE_DATUM,RETOURTYP_BK_S,FINANZ_VORSORGER,KBA13_SEG_MITTELKLASSE,KBA13_VORB_1_2,KBA05_SEG5,...,D19_VERSI_DATUM,KBA05_SEG2,D19_TELKO_OFFLINE_DATUM,KBA05_ALTER2,ALTERSKATEGORIE_GROB,KONSUMNAEHE,D19_GESAMT_ONLINE_QUOTE_12,KBA05_KRSHERST1,LP_LEBENSPHASE_FEIN,KBA13_SEG_WOHNMOBILE
count,140371.000000,135672.000000,140371.000000,140371.000000,191652.000000,188439.000000,191652.000000,140371.000000,140371.000000,135672.000000,...,191652.000000,135672.000000,191652.000000,135672.000000,191652.000000,145001.000000,143955.000000,135672.000000,188439.000000,140371.000000
mean,2.858226,2.764638,2.730386,2.615540,7.445714,3.716311,4.183317,2.981143,2.852861,1.784230,...,9.209171,2.921185,9.799339,2.797548,3.060907,3.129978,3.522879,3.153569,18.181571,2.669405
std,1.094689,1.145709,0.971710,0.931909,3.117772,1.113933,0.995365,0.973991,0.961431,1.295017,...,1.856680,1.090810,0.832611,1.138182,1.086254,1.439740,4.561253,1.102288,15.009985,1.404804
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
25%,2.000000,2.000000,2.000000,2.000000,5.000000,3.000000,3.000000,2.000000,2.000000,1.000000,...,10.000000,2.000000,10.000000,2.000000,3.000000,2.000000,0.000000,2.000000,0.000000,2.000000
50%,3.000000,3.000000,3.000000,3.000000,9.000000,3.000000,5.000000,3.000000,3.000000,2.000000,...,10.000000,3.000000,10.000000,3.000000,3.000000,3.000000,0.000000,3.000000,16.000000,3.000000
75%,4.000000,3.000000,3.000000,3.000000,10.000000,5.000000,5.000000,4.000000,3.000000,3.000000,...,10.000000,3.000000,10.000000,3.000000,4.000000,4.000000,10.000000,4.000000,36.000000,3.000000
max,5.000000,9.000000,5.000000,5.000000,10.000000,5.000000,5.000000,5.000000,5.000000,9.000000,...,10.000000,9.000000,10.000000,9.000000,9.000000,7.000000,10.000000,9.000000,40.000000,5.000000


### Now, let's check for missing values:

In [11]:
# Check for missing values in the azdias dataset
missing_azdias = azdias.isna().sum()
print(missing_azdias)

# Check for missing values in the customers dataset
missing_customers = customers.isna().sum()


KBA13_OPEL                    105800
KBA05_KW1                     133324
KBA13_KMH_180                 105800
KBA13_HALTER_25               105800
D19_GESAMT_ONLINE_DATUM            0
RETOURTYP_BK_S                  4854
FINANZ_VORSORGER                   0
KBA13_SEG_MITTELKLASSE        105800
KBA13_VORB_1_2                105800
KBA05_SEG5                    133324
D19_GESAMT_DATUM                   0
WOHNLAGE                       93148
PLZ8_BAUMAX                   116515
KBA13_SEG_SPORTWAGEN          105800
KBA13_KW_90                   105800
KBA13_SITZE_6                 105800
KBA13_SEG_MINIWAGEN           105800
KBA13_KRSSEG_KLEIN            105800
KBA05_SEG3                    133324
KBA13_HALTER_55               105800
KBA13_HALTER_45               105800
D19_VERSAND_DATUM                  0
KBA05_SEG4                    133324
KBA13_HALTER_66               105800
KBA05_ANTG3                   133324
KBA05_HERST5                  133324
SEMIO_FAM                          0
D

Where do we find a lot of missings? And how do we deal with them?

In [12]:
missing_customers_vec = missing_customers[missing_customers>0]/len(customers)
print(missing_customers_vec[missing_customers_vec > 0.3])

customers[missing_customers_vec[missing_customers_vec > 0.3].index].head()

Series([], dtype: float64)


""
0
1
2
3
4


In [13]:
missing_azdias_vec = missing_azdias[missing_azdias>0]/len(azdias)
missing_azdias_vec[missing_azdias_vec > 0.3]



Series([], dtype: float64)

Now let's start to impute the rest of the columns

In [14]:

#print(missing_customers_vec[missing_customers_vec <= 0.3])
customers[missing_customers_vec[missing_customers_vec <= 0.3].index].head()

,KBA13_OPEL,KBA05_KW1,KBA13_KMH_180,KBA13_HALTER_25,RETOURTYP_BK_S,KBA13_SEG_MITTELKLASSE,KBA13_VORB_1_2,KBA05_SEG5,WOHNLAGE,PLZ8_BAUMAX,...,KBA05_ALTER1,KBA13_KW_50,KBA05_ALTER3,KBA05_SEG2,KBA05_ALTER2,KONSUMNAEHE,D19_GESAMT_ONLINE_QUOTE_12,KBA05_KRSHERST1,LP_LEBENSPHASE_FEIN,KBA13_SEG_WOHNMOBILE
0,3.0,3.0,2.0,1.0,5.0,3.0,3.0,3.0,7.0,1.0,...,2.0,2.0,4.0,3.0,2.0,5.0,0.0,4.0,20.0,4.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,0.0,NaN,NaN,NaN
2,1.0,3.0,3.0,3.0,5.0,2.0,2.0,1.0,2.0,3.0,...,2.0,4.0,2.0,2.0,4.0,1.0,0.0,4.0,13.0,5.0
3,1.0,3.0,3.0,4.0,3.0,3.0,2.0,0.0,7.0,1.0,...,2.0,2.0,3.0,4.0,3.0,2.0,0.0,4.0,0.0,2.0
4,4.0,3.0,4.0,2.0,5.0,2.0,3.0,1.0,3.0,2.0,...,2.0,4.0,4.0,2.0,4.0,1.0,10.0,2.0,31.0,3.0


In [15]:
# 1.) KBA13 columns
# List of KBA13 columns 
kba13_columns = [col for col in customers.columns if col.startswith('KBA13_') and col != 'KBA13_ANZAHL_PKW']
# Impute NaNs with -1
customers[kba13_columns] = customers[kba13_columns].fillna(-1)

# Impute NaNs in 'KBA13_ANZAHL_PKW' with the mean value
mean_value = customers['KBA13_ANZAHL_PKW'].mean()
customers['KBA13_ANZAHL_PKW'].fillna(mean_value, inplace=True)

# 2.) KBA05 columns
# List of KBA05 columns 
kba05_columns = [col for col in customers.columns if col.startswith('KBA05_') and col not in ['KBA05_GBZ', 'KBA05_BAUMAX']]
customers[kba05_columns] = customers[kba05_columns].fillna(-1)
# 9 means the same as -1
customers[kba05_columns] = customers[kba05_columns].replace(9, -1)

kba05_special_treatment = ['KBA05_GBZ', 'KBA05_BAUMAX']
customers[kba05_special_treatment] = customers[kba05_special_treatment].fillna(-1)
# map 0 -> -1 for KBA05_GBZ KBA05_BAUMAX
customers[kba05_special_treatment] = customers[kba05_special_treatment].replace(0, -1)

# 3.) LP columns
# List of LP columns 
# Impute NaNs with -1 : LP_; D19_
nan_to_minus_1 =[
'MOBI_REGIO',
'ONLINE_AFFINITAET',
'GFK_URLAUBERTYP',
'OST_WEST_KZ',
'BALLRAUM',
'EWDICHTE',
'INNENSTADT',
'CAMEO_DEU_2015',
'CAMEO_DEUG_2015',
'ORTSGR_KLS9',
'GEBAEUDETYP_RASTER',
'WOHNLAGE',
'KONSUMNAEHE',
'RELAT_AB'
]

map_0_to_minus_1=[
'W_KEIT_KIND_HH',
'KKK',
'REGIOTYP',
'HH_EINKOMMEN_SCORE',
'CJT_GESAMTTYP',
'RETOURTYP_BK_S',
'TITEL_KZ',
'WOHNDAUER_2008',
'ALTER_HH',
'GEBAEUDETYP',
]

nan_to_minus_1 = nan_to_minus_1 + map_0_to_minus_1

lp_columns = [col for col in customers.columns if col in nan_to_minus_1 or col.startswith('LP_') or  col.startswith('D19_') or  col.startswith('PLZ8_') ]
customers[lp_columns] = customers[lp_columns].fillna(-1)

customers[map_0_to_minus_1] = customers[map_0_to_minus_1].replace(0, -1)

customers['RELAT_AB'] = customers['RELAT_AB'].replace(9, -1)


# 5.) impute numeric columns with mean
customers[['ANZ_PERSONEN',         
'ANZ_TITEL'      ,        
'MIN_GEBAEUDEJAHR' ,      
'ANZ_HAUSHALTE_AKTIV',    
'ARBEIT' ,                
'ANZ_HH_TITEL' ]] = customers[['ANZ_PERSONEN',        
'ANZ_TITEL'      ,        
'MIN_GEBAEUDEJAHR' ,      
'ANZ_HAUSHALTE_AKTIV',    
'ARBEIT' ,                
'ANZ_HH_TITEL' ]].fillna(customers[['ANZ_PERSONEN',           
'ANZ_TITEL'      ,        
'MIN_GEBAEUDEJAHR' ,      
'ANZ_HAUSHALTE_AKTIV',    
'ARBEIT' ,                
'ANZ_HH_TITEL' ]].mean())

#customers['ANZ_PERSONEN']

missing_customers = customers.isna().sum()
missing_customers[missing_customers >0].sort_values()


Series([], dtype: int64)

TODO: Specify DataTypes!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Check if there are rows with many missings:

In [16]:
missing_values_per_row = customers.isna().sum(axis=1)
print(missing_values_per_row[missing_values_per_row/len(customers) > 0.1])
missing_values_per_row = azdias.isna().sum(axis=1)
print(missing_values_per_row[missing_values_per_row/len(azdias) > 0.1])


Series([], dtype: int64)
Series([], dtype: int64)


In [17]:
customers.iloc[0:10]

,KBA13_OPEL,KBA05_KW1,KBA13_KMH_180,KBA13_HALTER_25,D19_GESAMT_ONLINE_DATUM,RETOURTYP_BK_S,FINANZ_VORSORGER,KBA13_SEG_MITTELKLASSE,KBA13_VORB_1_2,KBA05_SEG5,...,D19_VERSI_DATUM,KBA05_SEG2,D19_TELKO_OFFLINE_DATUM,KBA05_ALTER2,ALTERSKATEGORIE_GROB,KONSUMNAEHE,D19_GESAMT_ONLINE_QUOTE_12,KBA05_KRSHERST1,LP_LEBENSPHASE_FEIN,KBA13_SEG_WOHNMOBILE
0,3.0,3.0,2.0,1.0,10,5.0,5,3.0,3.0,3.0,...,10,3.0,10,2.0,4,5.0,0.0,4.0,20.0,4.0
1,-1.0,-1.0,-1.0,-1.0,9,-1.0,5,-1.0,-1.0,-1.0,...,10,-1.0,10,-1.0,4,5.0,0.0,-1.0,-1.0,-1.0
2,1.0,3.0,3.0,3.0,10,5.0,5,2.0,2.0,1.0,...,10,2.0,10,4.0,4,1.0,0.0,4.0,13.0,5.0
3,1.0,3.0,3.0,4.0,10,3.0,5,3.0,2.0,0.0,...,9,4.0,10,3.0,4,2.0,0.0,4.0,0.0,2.0
4,4.0,3.0,4.0,2.0,1,5.0,4,2.0,3.0,1.0,...,10,2.0,10,4.0,3,1.0,10.0,2.0,31.0,3.0
5,1.0,3.0,2.0,2.0,10,3.0,5,4.0,3.0,4.0,...,10,2.0,10,2.0,3,2.0,0.0,3.0,17.0,3.0
6,3.0,2.0,3.0,2.0,5,5.0,5,2.0,3.0,3.0,...,10,3.0,10,3.0,4,4.0,7.0,4.0,20.0,2.0
7,2.0,2.0,2.0,3.0,10,5.0,5,3.0,2.0,2.0,...,10,2.0,9,2.0,4,3.0,0.0,4.0,20.0,3.0
8,1.0,5.0,1.0,2.0,10,5.0,5,3.0,3.0,0.0,...,10,3.0,10,1.0,4,1.0,0.0,3.0,6.0,5.0
9,1.0,3.0,3.0,3.0,4,2.0,4,3.0,2.0,2.0,...,10,3.0,8,5.0,3,5.0,6.0,4.0,28.0,3.0


In [18]:
def preprocess_data(df):
    # remove all columns with NaN in more than 50% 
    df.drop(['ALTER_KIND1','ALTER_KIND2','ALTER_KIND3','ALTER_KIND4','EXTSEL992','KK_KUNDENTYP'], inplace=True)
    # remove all columns that are not described in the file.xlsx, because it is not possible to clarify sensibly how to deal with missing values here.  
    data = pd.read_excel("./DIAS Information Levels - Attributes 2017.xlsx")
    keep_columns = set(df.columns) & set(data.Attribute)
    df = df[keep_columns]
    
    ### IMPUTATION ###
    
    # 1.) KBA13 columns
    
    # List of KBA13 columns 
    kba13_columns = [col for col in df.columns if col.startswith('KBA13_') and col != 'KBA13_ANZAHL_PKW']
    # Impute NaNs with -1
    df[kba13_columns] = df[kba13_columns].fillna(-1)
    # Impute NaNs in 'KBA13_ANZAHL_PKW' with the mean value
    mean_value = df['KBA13_ANZAHL_PKW'].mean()
    df['KBA13_ANZAHL_PKW'].fillna(mean_value, inplace=True)

    # 2.) KBA05 columns
    # List of KBA05 columns 
    kba05_columns = [col for col in df.columns if col.startswith('KBA05_') and col not in ['KBA05_GBZ', 'KBA05_BAUMAX']]
    df[kba05_columns] = df[kba05_columns].fillna(-1)
    # 9 means the same as -1
    df[kba05_columns] = df[kba05_columns].replace(9, -1)

    kba05_special_treatment = ['KBA05_GBZ', 'KBA05_BAUMAX']
    df[kba05_special_treatment] = df[kba05_special_treatment].fillna(-1)
    # map 0 -> -1 for KBA05_GBZ KBA05_BAUMAX
    df[kba05_special_treatment] = df[kba05_special_treatment].replace(0, -1)

    # 3.) LP columns
    # List of LP columns 
    # Impute NaNs with -1 : LP_; D19_
    nan_to_minus_1 =[
    'MOBI_REGIO',
    'ONLINE_AFFINITAET',
    'GFK_URLAUBERTYP',
    'OST_WEST_KZ',
    'BALLRAUM',
    'EWDICHTE',
    'INNENSTADT',
    'CAMEO_DEU_2015',
    'CAMEO_DEUG_2015',
    'ORTSGR_KLS9',
    'GEBAEUDETYP_RASTER',
    'WOHNLAGE',
    'KONSUMNAEHE',
    'RELAT_AB'
    ]

    map_0_to_minus_1=[
    'W_KEIT_KIND_HH',
    'KKK',
    'REGIOTYP',
    'HH_EINKOMMEN_SCORE',
    'CJT_GESAMTTYP',
    'RETOURTYP_BK_S',
    'TITEL_KZ',
    'WOHNDAUER_2008',
    'ALTER_HH',
    'GEBAEUDETYP',
    ]

    nan_to_minus_1 = nan_to_minus_1 + map_0_to_minus_1

    lp_columns = [col for col in df.columns if col in nan_to_minus_1 or col.startswith('LP_') or  col.startswith('D19_') or  col.startswith('PLZ8_') ]
    df[lp_columns] = df[lp_columns].fillna(-1)

    df[map_0_to_minus_1] = df[map_0_to_minus_1].replace(0, -1)

    df['RELAT_AB'] = df['RELAT_AB'].replace(9, -1)


    # 5.) impute numeric columns with mean
    df[['ANZ_PERSONEN',         
    'ANZ_TITEL'      ,        
    'MIN_GEBAEUDEJAHR' ,      
    'ANZ_HAUSHALTE_AKTIV',    
    'ARBEIT' ,                
    'ANZ_HH_TITEL' ]] = df[['ANZ_PERSONEN',        
    'ANZ_TITEL'      ,        
    'MIN_GEBAEUDEJAHR' ,      
    'ANZ_HAUSHALTE_AKTIV',    
    'ARBEIT' ,                
    'ANZ_HH_TITEL' ]].fillna(df[['ANZ_PERSONEN',           
    'ANZ_TITEL'      ,        
    'MIN_GEBAEUDEJAHR' ,      
    'ANZ_HAUSHALTE_AKTIV',    
    'ARBEIT' ,                
    'ANZ_HH_TITEL' ]].mean())
    
    
    

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [19]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
